In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
# id=sys.argv[1]
id=209
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)
cart_products=pd.read_sql("select * from cart_products where quotation_id ="+str(id),cnx)

cliente=pd.read_sql("""select * from customers where customers.id="""+str(cotizacion['customer_id'].values[0]),cnx)
user=pd.read_sql("""select * from users where users.id="""+str(cotizacion['user_id'].values[0]),cnx)
productos=pd.read_sql("""select * from cart_products where cart_products.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
questionario=pd.read_sql("""select * from questionaries where questionaries.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
if(len(questionario)==0):
    questionario=pd.read_sql("""select * from questionaries limit 1""",cnx)

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
from tablas_dict import redact
from tablas_dict import extras
from tablas_dict import ref
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

print('producots en carrito ',len(cart_products))
products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    cart_reference=[]
    if(len(p)>0):
        cart_reference=cart_products.loc[cart_products['id']==p.cart_id.values[0]]
    if(('cost' not in p.columns)&(len(p)>0)&(len(cart_reference)>0)):
        
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        # if('total_kg' in p.columns):
        #     p=p.assign(cost=costo*p.total_kg)
        # if('total_weight' in p.columns):
        #     p=p.assign(cost=costo*p.total_weight)
        # if('weight_kg' in p.columns):
         
        #     p=p.assign(cost=costo*p.weight_kg)
        # if('weight' in p.columns):
          
        #     p=p.assign(cost=costo*p.weight)
        # if('long' in p.columns):
           
        #     p=p.assign(cost=costo*p.long)
    # if(len(p)>0)&(len(cart_reference)>0):
    if(len(p)>0):
        if(len(cart_reference)>0):
            print('ha sido encontrada una instancia de ',p)
            p=p.assign(cost_unit=cart_reference['unit_price'].values[0])
            p=p.assign(cost_total=cart_reference['total_price'].values[0])
            p=p.assign(cantidad=cart_reference['amount'].values[0])
            p=p.assign(sku=cart_reference['sku'].values[0])
        else:
            # p=p.assign(cost_unit=cart_reference['unit_price'].values[0])
            # p=p.assign(cost_total=cart_reference['total_price'].values[0])
            p=p.assign(cantidad=p['amount'].values[0])
    products=pd.concat([products,p],ignore_index=True)
# print(products)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
largo_cols=['long','length','length_meters','frame_background',
       'length_dimension']
ancho_cols=['uncut_front',  'uncut_background',
       'depth']
products[cols_kg+cols_m2+largo_cols+ancho_cols]=products[cols_kg+cols_m2+largo_cols+ancho_cols].fillna(0)
price_cols=['price','total_price','import','unit_price']
products[price_cols] =products[price_cols].fillna(0)    
pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')

if(cotizacion['type'].values[0]=='DOBLE PROFUNDIDAD'):
    doc= DocxTemplate("plantilla_d.docx")
else:
    if(cotizacion['type'].values[0]=='DRIVE IN'):
        doc= DocxTemplate("plantilla_drivein.docx")
    else:
        doc = DocxTemplate("plantilla.docx")



instalacion_tables=['quotation_installs','quotation_uninstalls']
productos=[]
print(products.columns)
for col in largo_cols:
    products[col]=products[col].astype(float)

for i in range(len(products)):
    this_color=' '
    seccion=None
    carga=None
    altura=0
    ancho=0
    print(products['tabla'].values[i],products['cantidad'].values[i],products['cost_unit'].values[i])
    if(products['tabla'].values[i] not in instalacion_tables):
        if('frames' in products['tabla'].values[i]):
            this_color='Color:  Galvanizado. (otros colores opcionales).'
            seccion=questionario['section'].values[0]
        if('joist' in products['tabla'].values[i]):
            this_color='Anaranjado'
        if(('brazo' in products['tabla'].values[i])|('arrios' in products['tabla'].values[i])):
            this_color='Anaranjado'
            carga='{0:.2f}'.format(products['weight_kg'].values[i])
        
        if(products['cantidad'].values[i]>0):
            print('        SI----')
            #Gerenerar los atributos escogidos para cada talba
            #atributos por defecto
            product_dict={'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':ref[products['tabla'].values[i]],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':int(products['cantidad'].values[i]),
                        'color': this_color,
                        'largo': round(products[largo_cols].sum(axis=1).values[i],2),
                        'carga': carga,
                        'altura': round(products[largo_cols].sum(axis=1).values[i],2),
                        'ancho': round(products[ancho_cols].sum(axis=1).values[i],2),
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':seccion,
                        'peralte':None,
                        'sku':products['sku'].values[i]}
            #marcos
            if('frames' in products['tabla'].values[i]):
                section="84mm por 38mm"
                if(products['model'].values[i]=='TC-2'):
                    section="84mm por 52mm"
                if(products['model'].values[i]=='TC-3'):
                    section="84mm por 72mm"
                product_dict={'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':products['model'].values[i],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':int(products['cantidad'].values[i]),
                        'color': 'Galvanizado (otros colores disponibles)',
                        'largo': 0,
                        'carga': 0,
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho':0,
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':section,
                        'peralte':None,
                        'sku':products['sku'].values[i]}
            if('structural_frames' in products['tabla'].values[i]):
                product_dict={'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':'ME-'+str(products['model'].values[i][0]),
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':int(products['cantidad'].values[i]),
                        'color': 'Azul TYRSA (otros colores disponibles)',
                        'largo': 0,
                        'carga': 0,
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho': 0,
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'peralte':None,
                        'seccion':products['model'].values[i],
                        'sku':products['sku'].values[i]}
            #vigas
            if('joist' in products['tabla'].values[i]):
                product_dict={'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':ref[products['tabla'].values[i]],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':int(products['cantidad'].values[i]),
                        'color': this_color,
                        'largo': products[largo_cols].sum(axis=1).values[i],
                        'carga': 'loading_capacity',
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho': products[ancho_cols].sum(axis=1).values[i],
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':None,
                        'peralte':str(round(products['camber'].values[i],1))+'"(in)',
                        'sku':products['sku'].values[i]}
            #brazos
            if(('brazo' in products['tabla'].values[i])|('arrios' in products['tabla'].values[i])):
                product_dict={'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':ref[products['tabla'].values[i]],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':int(products['cantidad'].values[i]),
                        'color': this_color,
                        'largo': products[largo_cols].sum(axis=1).values[i],
                        'carga': carga,
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho': products[ancho_cols].sum(axis=1).values[i],
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':None,
                        'peralte':None,
                        'sku':products['sku'].values[i]}
            
            productos.append(product_dict)

print('X-X-X-X-X-X-X calculado el total',products[price_cols]) 

precio_total=cart_products['total_price'].sum()
print(precio_total)
kilos_totales=products[cols_kg].sum(axis=1).sum()
fletes_tables=['packagings','quotation_travel_assignments']
instalacion_tables=['quotation_installs','quotation_uninstalls']
precios=products[price_cols+['tabla','print']]
costo_flete=precios.loc[precios['tabla'].isin(fletes_tables)].sum(axis=1,numeric_only=True).sum()
costo_instalacion=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='Sí')].sum(axis=1,numeric_only=True).sum()
costo_instalacion_incluida=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='In')].sum(axis=1,numeric_only=True).sum()

if(costo_instalacion>0):
    print('la instalacion se desglosa')
    print(precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='Sí')])
    des_inst=1
else:
    des_inst=0
text=str(questionario['ndib'].values[0])+','
dibujos=[]
while(',' in text):
    print('entra iteracion')
    myindex=text.index(',')
    print(myindex)
    dibujos.append(text[0:myindex])
    text=text[myindex+1:]

if(len(dibujos)>0):
    primer_dibujo=dibujos[0]
else:
    primer_dibujo=' '
# try:
#     photo=InlineImage(doc,'storage/img/'+str(id).zfill(8)+'img0.JPG',width=Mm(50))
# except:
#     photo=''
photos = []
for i in range(10):
    path = f"storage/img/{str(id).zfill(8)}img{i}.jpg"
    if os.path.exists(path):  
        photos.append(InlineImage(doc, path, width=Mm(50)))
    else:
        photos.append("")  # No existe la imagen

context={
    'cliente':cliente['customer'].values[0],
    'direccion':cliente['address'].values[0]+' '+cliente['outdoor'].values[0]+', '+cliente['city'].values[0]+' '+cliente['suburb'].values[0]+' '+cliente['state'].values[0]+', cp: '+str(cliente['zip_code'].values[0]),
    'folio': cotizacion['invoice'].values[0],
    'fecha': today,
    'asesor': user['name'].values[0],
    'mayus_type':cotizacion['type'].values[0],
    'type':cotizacion['type'].values[0].capitalize(),
    'productos': productos,
    'precio_total': '{:2,.2f}'.format(precio_total),
    'kilos_totales': '{0:.2f}'.format(kilos_totales),
    'costo_flete':'{0:.2f}'.format(costo_flete),
    'costo_instalacion':'{:2,.2f}'.format(costo_instalacion),
    'costo_instalacion_incluida':'{:2,.2f}'.format(costo_instalacion_incluida),
    'costo_selectivo':'{:2,.2f}'.format(precio_total - costo_flete -costo_instalacion),
    'estado': cliente['state'].values[0],
    'a5': questionario['a5'].values[0], #que productos e almacena
   #reativos de nivel
    'a8': questionario['a8'].values[0],#frente
    'a9': questionario['a9'].values[0],#fondo
    'a10': questionario['a10'].values[0],#alto
    'a11': questionario['a11'].values[0],#peso
# dimensiones de la tarima
    'a18': questionario['a18'].values[0],#frente
    'a19': questionario['a19'].values[0],#fondo
    'a20': questionario['a20'].values[0],#alto
    'a21': questionario['a21'].values[0],#peso
#ambiente
    'a25': questionario['a25'].values[0], #temperatura
    'a26': questionario['a26'].values[0], #inflamable
    'a27': questionario['a27'].values[0], #explosivo
    'a28': questionario['a28'].values[0], #corrosivo
#Reactivos extra
#     'ndib':  str(questionario['ndib'].values[0]).replace(',',"""
# """), #numero de dibujos
'ndib':  str(questionario['ndib'].values[0]).replace(',','O'),
'primer_dibujo':primer_dibujo ,
'photos':photos,
    'npos':  questionario['npos'].values[0], #numero de posiciones
    'vigas':  questionario['vigas'].values[0], #vigas
    'tiempo':  questionario['tiempo'].values[0], #tiempo de entrega
    'dibujos': dibujos,
    'des_inst': des_inst
    }
doc.render(context) 
doc.save("storage/Cotizacion"+str(id)+".docx")


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

producots en carrito  8


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

ha sido encontrada una instancia de     id  quotation_id  amount            type  depth  developing   long caliber  \
0  11           209       3  CROSBAR 1.06 M   1.06        0.15  1.063      14   

   kg_m2  weight    m2  connector           sku  unit_price  total_price  \
0  15.25  2.4248  0.32        0.0  TC0000115603      277.52       832.56   

           created_at          updated_at  cart_id                tabla  
0 2025-11-04 12:19:14 2025-11-04 12:19:16      176  selective_crossbars  
ha sido encontrada una instancia de     id  quotation_id  amount model caliber  depth  total_load_kg total_poles  \
0  43           209       4   TC0      14   0.91          136.0           8   

  total_crossbars total_diagonals  ... total_kg  total_m2           sku  \
0               8               0  ...    38.69      4.42  TC0000000001   

  unit_price  total_price          created_at          updated_at cart_id  \
0     855.77      6019.84 2025-10-28 12:23:06 2025-11-03 16:49:47     172  

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

ha sido encontrada una instancia de     id  quotation_id  amount                    use  developing  long caliber  \
0   9           209       3  DISTANCIADOR DE 50 MM       0.084  0.15      12   

   kg_m2    m2           sku  unit_price  total_price          created_at  \
0  21.25  0.03  TC0000115584       38.63       167.18 2025-10-28 15:07:51   

           updated_at  cart_id              tabla  
0 2025-10-28 15:08:52      168  selective_spacers  
ha sido encontrada una instancia de     id  quotation_id  amount model caliber  total_load_kg total_poles  \
0   4           209       4   3IN    EST3         1776.0           8   

  total_crossbars total_diagonals total_plates  total_kg  total_m2  \
0               8               0            8     76.52      3.64   

            sku  total_price          created_at          updated_at  cart_id  \
0  TC0000003813     14335.76 2025-11-04 11:46:41 2025-11-04 13:39:04      179   

   depth  length                        tabla  
0   0.91 

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

In [25]:
round(1.030000333,3)

1.03

In [4]:
'{:2,.2f}'.format(89897)

'89,897.00'

In [29]:
products[['tabla','amount']+price_cols]

,tabla,amount,price,total_price,import,unit_price
0,packagings,5,0,0.00,256.96,0.000
1,quotation_installs,100,0,0.00,51285.00,0.000
2,quotation_travel_assignments,10,0,0.00,20730.00,0.000
3,selective_heavy_load_frames,50,0,6457.10,0.00,0.000
4,selective_joist_c2_s,200,0,37010.79,0.00,178.220
5,quotation_drive_in_soportes,60,0,411.84,0.00,6.864
6,quotation_drive_in_arriostrados,50,0,3900.00,0.00,78.000
8,quotation_drive_in_brazos,200,0,25080.00,0.00,125.400
9,quotation_drive_in_brazos,60,0,8523.00,0.00,142.050
10,quotation_drive_in_brazos,80,0,14013.60,0.00,175.170


In [14]:
(num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i])/products['amount'].values[i]

    

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_360\3956706895.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  (num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i])/products['amount'].values[i]


inf

In [ ]:
38

9.5

In [31]:
products.reset_index(drop=True)

,id,quotation_id,amount,type,depth,developing,long,caliber,kg_m2,weight,...,dias,operarios,breakdown_uninstall,uncut_front,cut_front,uncut_background,cut_background,desarrollo,piezas_nec,piece_weight
0,1,152,5,NaN,NaN,NaN,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,152,100,INSTALACION,NaN,NaN,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,152,10,NaN,NaN,NaN,NaN,NaN,NaN,0.00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,152,50,NaN,NaN,NaN,NaN,12,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,152,200,NaN,NaN,NaN,NaN,12,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,152,60,NaN,NaN,NaN,NaN,14,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,152,50,NaN,NaN,NaN,NaN,12,NaN,21.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.76,1.0,2.8300
7,27,152,200,NaN,NaN,NaN,NaN,14,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.1371
8,28,152,60,NaN,NaN,NaN,NaN,EST 3 IN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0988
9,29,152,80,NaN,NaN,NaN,NaN,EST 4 IN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5838


In [27]:

calest3.columns

Index(['Unnamed: 0', 'SKU', 'PESO (KG)', 'M2', 'PRECIO'], dtype='object')

In [3]:
query="""insert into drive_in_guias(length,weight, m2, sku) values(%s,%s,%s,%s)"""
for i in range(len(guias_data)):
    cursor.execute(query,tuple([

        str(guias_data['Unnamed: 0'].values[i]),
        str(guias_data['PESO (KG)'].values[i]),
        str(guias_data['M2'].values[i]),
        str(guias_data['SKU'].values[i])
       
    ]))

In [4]:
cnx.commit()

LAMINA  ESTRUCTURAL CALIBRE EST 3 IN 33 3
LAMINA  ESTRUCTURAL CALIBRE EST 6 IN 34 4


In [31]:
cnx.commit()

In [4]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
id=209
# id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
a_lite='#b4c7ed'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
# join para cobros
quotation=pd.read_sql("select * from quotations where id=" +str(id),cnx)

cart_products=pd.read_sql("select * from cart_products where quotation_id ="+str(id),cnx)
writer = pd.ExcelWriter('storage/report/administrativo'+str(id)+'.xlsx', engine='xlsxwriter')

workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1,
    'font_size':13})
blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
blue_footer_format_bold_kg = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '#,##0.00',
    'font_size':11})

red_header_format = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})

red_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '0.00'})
blue_content_lite = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color':a_lite,
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit_lite = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color':a_lite,
    'border_color':a_color,
    'font_size':10,
    'num_format': '0.00'})
blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})
df=quotation[['id','user_id']]

df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    print(i)
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    cart_reference=[]
    if(len(p)>0):
        cart_reference=cart_products.loc[cart_products['id']==p.cart_id.values[0]]
    if(('cost' not in p.columns)&(len(p)>0)&(len(cart_reference)>0)):
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        print(str(p['caliber'].values[0]))
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        if('total_kg' in p.columns):
            p=p.assign(cost=costo*p.total_kg)
        if('total_weight' in p.columns):
           
            p=p.assign(cost=costo*p.total_weight)
        if('weight_kg' in p.columns):
         
            p=p.assign(cost=costo*p.weight_kg)
        if('weight' in p.columns):
          
            p=p.assign(cost=costo*p.weight)
        if('long' in p.columns):
              
            p=p.assign(cost=costo*p.long)

        print(i)
    
    if(len(p)>0)&(len(cart_reference)>0):
        try:
            p=p.assign(cost_unit=cart_reference['costo_sn_factor'].values[0]/cart_reference['amount'].values[0])
            p=p.assign(cost_total=cart_reference['costo_sn_factor'].values[0])
            p=p.assign(cantidad=cart_reference['amount'].values[0])
        except:
            print('nothing')
    products=pd.concat([products,p],ignore_index=True)

products=products.loc[products['amount']>0].reset_index(drop=True)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
price_cols=['price','total_price','import','unit_price']
products[cols_kg+cols_m2+price_cols+['amount']]=products[cols_kg+cols_m2+price_cols+['amount']].fillna(0)
largo_cols=['long','length','length_meters','frame_background',
       'length_dimension']
ancho_cols=['uncut_front',  'uncut_background',
       'depth']

pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'REPORTE POR COTIZACION ', negro_b)
worksheet.merge_range('B3:F3', 'ADMINISTRATIVO', negro_s)
worksheet.merge_range('B4:F4', 'COSTOS ', negro_b)
worksheet.write('H2', 'AÑO', negro_b)

worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)

worksheet.write('L2', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
worksheet.merge_range('A6:A8', 'PDA', blue_header_format)
worksheet.merge_range('B6:B8', 'SKU	', blue_header_format)
worksheet.merge_range('C6:C8', 'CANT', blue_header_format)	
worksheet.merge_range('D6:D8', 'DESCRIPCION', blue_header_format)	
worksheet.merge_range('E6:E8', 'CTO UNIT', blue_header_format)	
worksheet.merge_range('F6:F8', 'CTO TOTAL', blue_header_format)	
worksheet.merge_range('G6:G8', 'CALIBRE	', blue_header_format)
worksheet.merge_range('H6:H8', 'KG UNIT	', blue_header_format)
worksheet.merge_range('I6:I8', 'KG TOTAL', blue_header_format)	
worksheet.merge_range('J6:J8', 'CTO X KG', blue_header_format)	
worksheet.merge_range('K6:K8', 'M2 UNIT	', blue_header_format)
worksheet.merge_range('L6:L8', 'M2 TOT', blue_header_format)

row_count=9
def ret_na(value):
    try:
        x=float(value)
    except:
        x='NA'
    try:
        if(np.isnan(x)):
            x='NA'
        if(np.isinf(x)):
            x='NA'
    except:
        x='NA'
    return x
def num(value):
    try:
        x=float(value)
    except:
        x=0
    if((np.isnan(x))|(np.isinf(x))):
        x=0
    return x
#iterar sobre los productos

for i in range(0,len(products)):
    
    def my_func(row, table_name):
        return row in table_name
    piezas=materials.loc[materials['product'].apply(my_func,table_name=products['tabla'].values[i])]
    costo_product=products['cost'].values[i]
    n=len(piezas)
    
    print(n,products['tabla'].values[i],row_count,products['cost'].values[i])
    #Cosntruir la redaccion del producto
    if('joist' in products['tabla'].values[i]):
        peralte=' Peralte:'+str(products['camber'].values[i])
    else:
        peralte=' CALIBRE'+str(products['caliber'].values[i])

    if(products[largo_cols].sum(axis=1).values[i]>0):
        largo=' ALTURA: '+str(products[largo_cols].sum(axis=1).values[i])+'M'
    else: largo=''
    if(products[ancho_cols].sum(axis=1).values[i]>0):
        ancho= ' ANCHO: '+str(products[ancho_cols].sum(axis=1).values[i])+'M'
        if('frame' in products['tabla'].values[i]):
            ancho= ' FONDO: '+str(products[ancho_cols].sum(axis=1).values[i])+'M'
    else: ancho=''
    print('GENERANDO LA DESCRIPCION::-----:::---:::---')
    print(largo,ancho,peralte)
    descripcion=tablas[products['tabla'].values[i]]+products['protector'].values[i]+' '+products['model'].values[i]+peralte+largo+ancho
    if(i%2==1):
        formato=blue_content
        formato_unit=blue_content_unit
    if(i%2==0):
        formato=blue_content_lite
        formato_unit=blue_content_unit_lite
    
    #pda
    worksheet.write('A'+str(row_count), str(i*n+1), formato)
    #sku
    worksheet.write('B'+str(row_count), products['sku'].values[i], formato)
    worksheet.write('C'+str(row_count), str(products['amount'].values[i]), formato_unit)
    #descripcion
    worksheet.write('D'+str(row_count), descripcion, formato)
    #costos
    print(costo_product)
    if(products['total_price'].values[i]>0):
        precio_total=products['total_price'].values[i]
        if(products['unit_price'].values[i]>0):
            precio_unitario=products['unit_price'].values[i]
        else:
             precio_unitario=precio_total/products['amount'].values[i]
    else:
        precio_total=products[price_cols].sum(axis=1, numeric_only=True)[i]
        precio_unitario=products['amount'].values[i]*products[price_cols].sum(axis=1, numeric_only=True)[i]
    if('cost_total' in products):
        if(products['cost_total'].values[i]):
            precio_total=products['cost_total'].values[i]
            precio_unitario=products['cost_unit'].values[i]
    worksheet.write('E'+str(row_count), ret_na(precio_unitario), formato)
    worksheet.write('F'+str(row_count), ret_na(precio_total), formato)
    #calibre
    worksheet.write('G'+str(row_count), str(ret_na(products['caliber'].values[i])), formato)
    #pesos
    worksheet.write('H'+str(row_count),(num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i])/products['amount'].values[i], formato_unit)
    worksheet.write('I'+str(row_count),(num(products['total_weight'].values[i])+num(products['total_kg'].values[i])+products['weight'].values[i]+products['weight_kg'].values[i]), formato_unit)
    try: 
        worksheet.write('J'+str(row_count),(num(products['amount'].values[i]*products['cost'].values[i])/(num(products['total_weight'].values[i])+num(products['total_kg'].values[i]))), formato)
    except:
        worksheet.write('J'+str(row_count),0, formato)
    #medidas
    worksheet.write('K'+str(row_count),products['m2'].values[i]+products['total_m2'].values[0], formato_unit)
    worksheet.write('L'+str(row_count),(products['m2'].values[i]+products['total_m2'].values[0])*products['amount'].values[i], formato_unit)
    row_count=row_count+1
    #PIEZAS PIEZAS PIEZAS CICLO DE PIEZAS
    for j in range(0,n):
        print('entre al ciclo')
        print(piezas['cost'].fillna(0).values[j],piezas['amount'])
        costo= piezas['cost'].fillna(0).values[j].sum()
        cant= piezas['amount'].fillna(0).values[j].sum()*products['amount'].values[i]
        worksheet.write('A'+str(row_count), str(i*n+2+j), formato)
        #sku
        worksheet.write('B'+str(row_count), ''.join(materials['sku'].fillna('').values[0]), formato)
        worksheet.write('C'+str(row_count), str(cant), formato)
        worksheet.write('D'+str(row_count), str(piezas['description'].fillna('').values[j][0])+str(piezas['piece'].fillna('').values[j]), formato)
        #costos
        worksheet.write('E'+str(row_count),costo, formato)
        worksheet.write('F'+str(row_count), cant*costo, formato)
        #calibre
        worksheet.write('G'+str(row_count), piezas['type'].values[j][0]+piezas['type'].values[j][1], formato_unit)
        #pesos
        worksheet.write('H'+str(row_count),str(0.0), formato_unit)
        worksheet.write('I'+str(row_count),str(0.0), formato_unit)
        worksheet.write('J'+str(row_count), 0.0, formato)
        #medidas
        worksheet.write('K'+str(row_count), 0.0, formato_unit)
        worksheet.write('L'+str(row_count), 0.0, formato_unit)
        row_count=row_count+1
trow=row_count


#TOTALES
worksheet.merge_range('A'+str(trow+1)+':C'+str(trow), 'TOTAL (EQV M.N)', blue_header_format_bold)
worksheet.write_formula('F'+str(trow),'{=SUM(F9:F'+str(trow-1)+')}',blue_footer_format_bold)
worksheet.write_formula('I'+str(trow),'{=SUM(I9:I'+str(trow-1)+')}',blue_footer_format_bold_kg)
worksheet.write_formula('L'+str(trow),'{=SUM(L9:L'+str(trow-1)+')}',blue_footer_format_bold_kg)

# worksheet.write('K'+str(trow), str(cobros['amount'].sum()), blue_content)
# worksheet.write('L'+str(trow), str(cobros['exchange_sell'].values[0]*cobros['amount'].sum()), blue_content_bold)

#RESUMEN
worksheet.merge_range('B'+str(trow+3)+':C'+str(trow+4),'RESUMEN DE KILOS',blue_header_format_bold)
#subtabla 1, kilos
worksheet.write('B'+str(trow+5),'KILOS',blue_header_format)
worksheet.write('C'+str(trow+5),'CALIBRE',blue_header_format)
suma_peso=0
art_i=0
for i in products['caliber'].fillna('NA').astype(str).unique():
    if(i!='NA'):
        print(i)
        p=products.loc[products['caliber']==i]
        sum_kg=p['weight'].fillna(0).sum()+p['total_weight'].fillna(0).sum()+p['weight_kg'].fillna(0).sum()+p['total_kg'].fillna(0).sum()
        suma_peso=suma_peso+sum_kg
        worksheet.write('B'+str(trow+6+art_i),sum_kg,blue_content)
        worksheet.write('C'+str(trow+6+art_i),i,blue_content)
        art_i=art_i+1

worksheet.write('B'+str(trow+5+len(products['caliber'].unique())),suma_peso,blue_footer_format_bold_kg)
#subtabla2 costos F-G-H-I-J-K-L-M-N
#                     F-G-H-I-J-K-L-M-N
worksheet.merge_range('F'+str(trow+4)+':I'+str(trow+4),'RESUMEN DE COSTOS',blue_header_format_bold)
worksheet.write('J'+str(trow+4),'POSICION',blue_header_format)
worksheet.write('K'+str(trow+4),'KILOS',blue_header_format)
worksheet.write('L'+str(trow+4),'PORCENTAJE',blue_header_format)

worksheet.merge_range('F'+str(trow+5)+':I'+str(trow+5),'CONTRATO UNITARIO SOLO MATERIALES',blue_header_format)
worksheet.merge_range('F'+str(trow+6)+':I'+str(trow+6),'CONTRATO UNITARIO SOLO ARMADO',blue_header_format)
worksheet.merge_range('F'+str(trow+7)+':I'+str(trow+7),'CONTRATO UNITARIO SOLO TRASLADO',blue_header_format)
worksheet.merge_range('F'+str(trow+8)+':I'+str(trow+8),'CONTRATO UNITARIO COMBINADO',blue_header_format)


costo_total=products['cost'].sum()+materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()

worksheet.write('J'+str(trow+5),materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum(),blue_content)
worksheet.write('K'+str(trow+5),0,blue_content_unit)
worksheet.write('L'+str(trow+5),materials['cost'].fillna(0).sum(axis=1, numeric_only=True).sum()/costo_total*100,blue_content_unit)

worksheet.write('J'+str(trow+6),products.loc[products['tabla'].isin(['quotation_installs','quotation_uninstalls']),'cost'].sum(),blue_content)
worksheet.write('K'+str(trow+6),products.loc[products['tabla'].isin(['quotation_installs','quotation_uninstalls']),'cost'].sum()/costo_total*100,blue_content_unit)
worksheet.write('L'+str(trow+6),0,blue_content_unit)

worksheet.write('J'+str(trow+7),products.loc[(products['tabla'].isin(['quotation_travel_assignments','packagings'])),'cost'].sum(),blue_content)
worksheet.write('K'+str(trow+7),0,blue_content_unit)
worksheet.write('L'+str(trow+7),products.loc[(products['tabla'].isin(['quotation_travel_assignments','packagings'])),'cost'].sum()/costo_total*100,blue_content_unit)

worksheet.write('J'+str(trow+8),products['cost'].sum(),blue_content)
worksheet.write('K'+str(trow+8),products[cols_kg].sum(axis=1, numeric_only=True).sum(),blue_content_unit)
worksheet.write('L'+str(trow+8),'100%',blue_content)
#TODO: calcular bien esto, to6al menos iva


worksheet.set_column('B:B',20)
worksheet.set_column('D:D',30)
worksheet.set_column('J:J',15)
worksheet.set_column('E:E',15)
worksheet.set_column('F:F',15)
worksheet.set_column('G:L',15)
worksheet.set_column('N:R',15)
#worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

double_deep_crossbars
double_deep_floors
double_deep_floor_reinforcements
double_deep_heavy_load_frames
double_deep_joist_box25s
double_deep_joist_box25_caliber14s
double_deep_joist_box2s
double_deep_joist_box2_caliber14s
double_deep_joist_c2_s
double_deep_joist_chairs
double_deep_joist_l25_caliber14s
double_deep_joist_l25_s
double_deep_joist_l2_caliber14s
double_deep_joist_l2_s
double_deep_joist_lrs
double_deep_joist_structurals
double_deep_miniature_frames
double_deep_spacers
double_deep_structural_frames
freights
grills
installations
packagings
quot25_j_galvanized_panels
quot25_j_painted_panels
quot2_j_galvanized_panels
quot2_j_painted_panels
quotation_administratives
quotation_installs
quotation_protectors
quotation_specials
quotation_escuadras
quotation_gangplank_angles
quotation_respaldos
quotation_travel_assignments
quotation_uninstalls


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

quot_chair_j_galvanized_panels
quot_chair_j_painted_panels
selective_crossbars
selective_floors
selective_floor_reinforcements
selective_heavy_load_frames
14
selective_heavy_load_frames
selective_joist_box25s
selective_joist_box25_caliber14s
selective_joist_box2s
selective_joist_box2_caliber14s
selective_joist_c2_s
selective_joist_chairs
selective_joist_l25_caliber14s
selective_joist_l25_s
selective_joist_l2_caliber14s
14
selective_joist_l2_caliber14s
selective_joist_l2_s
selective_joist_lrs
selective_joist_structurals


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

selective_miniature_frames
selective_spacers
12
selective_spacers
selective_structural_frames
uninstalls
wood
quotation_drive_in_soportes
quotation_drive_in_guias
quotation_drive_in_arriostrados
quotation_drive_in_brazos
quotation_galletas
3 selective_heavy_load_frames 9 696.42
GENERANDO LA DESCRIPCION::-----:::---:::---
 ALTURA: 1.0M  FONDO: 0.91M  CALIBRE14
696.42
entre al ciclo
[1. 0.] 0    4
1    4
2    4
Name: amount, dtype: int64
entre al ciclo
[8. 0.] 0    4
1    4
2    4
Name: amount, dtype: int64
entre al ciclo
[ 0.  26.5] 0    4
1    4
2    4
Name: amount, dtype: int64
1 selective_joist_l2_caliber14s 13 21.599999999999998
GENERANDO LA DESCRIPCION::-----:::---:::---
 ALTURA: 0.02M   Peralte:3.0
21.599999999999998
entre al ciclo
[1. 0.] 4    2
Name: amount, dtype: int64
2 selective_spacers 15 2.6325
GENERANDO LA DESCRIPCION::-----:::---:::---
 ALTURA: 0.15M   CALIBRE12
2.6325
entre al ciclo
[1. 0.] 16    4
28    4
Name: amount, dtype: int64
entre al ciclo
[1. 0.] 16    4
28    

In [ ]:
products['sku'].fillna(0)!

0                
1                
2    TC0000117115
3                
4    TC0000117250
5    TC0000117250
6                
7    TC0000001171
8                
9                
Name: sku, dtype: object

In [27]:
products.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 8, 9, 10], dtype='int64')

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
id=sys.argv[1]
# id=82
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
# cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_18712\92736471.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)


DatabaseError: Execution failed on sql 'select * from quotations where id =--f=c:\Users\hp\AppData\Roaming\jupyter\runtime\kernel-v35871e93d9c1095680ef795974a6de72ad743e142.json': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near ':\Users\hp\AppData\Roaming\jupyter\runtime\kernel-v35871e93d9c1095680ef795974...' at line 1

In [6]:
#traer de vuelta info de marcos
pd.read_sql("select * from frames where model = "+"'TC3'"+"AND caliber = 14 and weight >=10000",cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_18712\1470919396.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select * from frames where model = "+"'TC3'"+"AND caliber = 14 and weight >=10000",cnx)


,id,caliber,buckling,weight,model,created_at,updated_at
0,36,14,1.22,13970,TC3,2022-04-16 19:34:02,2022-04-16 19:34:02
1,40,14,1.52,13010,TC3,2022-04-16 19:34:02,2022-04-16 19:34:02
2,44,14,1.83,12700,TC3,2022-04-16 19:34:02,2022-04-16 19:34:02
3,48,14,2.13,11880,TC3,2022-04-16 19:34:02,2022-04-16 19:34:02
4,52,14,2.44,10880,TC3,2022-04-16 19:34:02,2022-04-16 19:34:02
